In [1]:
import pandas as pd
import numpy as np

In [2]:
crime = pd.read_csv('crime.csv')
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


1. Data fields
2. Dates - timestamp of the crime incident
3. Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
4. Descript - detailed description of the crime incident (only in train.csv)
5. DayOfWeek - the day of the week
6. PdDistrict - name of the Police Department District
7. Resolution - how the crime incident was resolved (only in train.csv)
8. Address - the approximate street address of the crime incident 
9. X - Longitude
10. Y - Latitude

Problem: predict a crime category

## EDA

In [3]:
crime.describe(include='all')

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
count,878049,878049,878049,878049,878049,878049,878049,878049.000000,878049.000000
unique,389257,39,879,7,10,17,23228,NaN,NaN
top,2011-01-01 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,SOUTHERN,NONE,800 Block of BRYANT ST,NaN,NaN
freq,185,174900,60022,133734,157182,526790,26533,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.422616,37.771020
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030354,0.456893
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.513642,37.707879
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.432952,37.752427
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.416420,37.775421
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.406959,37.784369


In [4]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


There are a total of **878049 incidents** recorded and **389357 incidents that happened on unique dates** meaning some days had multiple incidents.

**Larceny/theft from locked auto** is the **most occuring crime** category with a total  of 174900 incidents out of 878049 and **800 Block of Bryant st** is the **top spot** for crimes.

The crimes mostly happen on **fridays** as well.

## Feature Engineering

Separate the date column into year and month

In [5]:
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [6]:
crime['year'] = crime['Dates'].str.split('-', expand=True)[0]
# crime.head()

In [7]:
crime['month'] = crime['Dates'].str.split('-',expand=True)[1]
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,year,month
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,05
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,05
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,05
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,05
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,05


In [8]:
crime = crime.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,year,month
0,2015-05-13 23:53:00,warrants,warrant arrest,wednesday,northern,"arrest, booked",oak st / laguna st,-122.425892,37.774599,2015,05
1,2015-05-13 23:53:00,other offenses,traffic violation arrest,wednesday,northern,"arrest, booked",oak st / laguna st,-122.425892,37.774599,2015,05
2,2015-05-13 23:33:00,other offenses,traffic violation arrest,wednesday,northern,"arrest, booked",vanness av / greenwich st,-122.424363,37.800414,2015,05
3,2015-05-13 23:30:00,larceny/theft,grand theft from locked auto,wednesday,northern,none,1500 block of lombard st,-122.426995,37.800873,2015,05
4,2015-05-13 23:30:00,larceny/theft,grand theft from locked auto,wednesday,park,none,100 block of broderick st,-122.438738,37.771541,2015,05


In [9]:
crime['Category'].value_counts()

Category
larceny/theft                  174900
other offenses                 126182
non-criminal                    92304
assault                         76876
drug/narcotic                   53971
vehicle theft                   53781
vandalism                       44725
warrants                        42214
burglary                        36755
suspicious occ                  31414
missing person                  25989
robbery                         23000
fraud                           16679
forgery/counterfeiting          10609
secondary codes                  9985
weapon laws                      8555
prostitution                     7484
trespass                         7326
stolen property                  4540
sex offenses forcible            4388
disorderly conduct               4320
drunkenness                      4280
recovered vehicle                3138
kidnapping                       2341
driving under the influence      2268
runaway                          1946
liq

In [10]:
crime['Category'].unique()

array(['warrants', 'other offenses', 'larceny/theft', 'vehicle theft',
       'vandalism', 'non-criminal', 'robbery', 'assault', 'weapon laws',
       'burglary', 'suspicious occ', 'drunkenness',
       'forgery/counterfeiting', 'drug/narcotic', 'stolen property',
       'secondary codes', 'trespass', 'missing person', 'fraud',
       'kidnapping', 'runaway', 'driving under the influence',
       'sex offenses forcible', 'prostitution', 'disorderly conduct',
       'arson', 'family offenses', 'liquor laws', 'bribery',
       'embezzlement', 'suicide', 'loitering',
       'sex offenses non forcible', 'extortion', 'gambling', 'bad checks',
       'trea', 'recovered vehicle', 'pornography/obscene mat'],
      dtype=object)

In [11]:
crime['year'].value_counts()

year
2013    75606
2014    74766
2003    73902
2004    73422
2012    71731
2005    70779
2008    70174
2006    69909
2009    69000
2007    68015
2011    66619
2010    66542
2015    27584
Name: count, dtype: int64

In [12]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
 9   year        878049 non-null  object 
 10  month       878049 non-null  object 
dtypes: float64(2), object(9)
memory usage: 73.7+ MB


In [13]:
# change year and month dtype to int
crime[['year','month']] =crime[['year','month']].astype('int64') 
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
 9   year        878049 non-null  int64  
 10  month       878049 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 73.7+ MB


In [16]:
crime[['year','month']].min()

year     2003
month       1
dtype: int64

In [17]:
crime[['year','month']].max()

year     2015
month      12
dtype: int64

These crimes occurred between 2003 and 2015

#### For each month, how many crimes occured?

In [22]:
# for each month, how many crimes occured
crime.groupby('month').agg({'Category':'count'}).sort_values(by='Category', ascending=False)

,Category
month,
10,80274
5,79644
4,78096
3,76320
1,73536
11,72975
9,71982
6,70892
2,70813


October has most number of crimes and December has the least number of crimes

#### For each day of the week, how many crimes occured?

In [23]:
# for each day of the week, how many crimes occured
crime.groupby('DayOfWeek').agg({'Category':'count'}).sort_values(by='Category', ascending=False)

,Category
DayOfWeek,
friday,133734
wednesday,129211
saturday,126810
thursday,125038
tuesday,124965
monday,121584
sunday,116707


Friday tops the list as the most violent day. Sunday is chiller lol

#### which police department district has the most crimes?

In [24]:

crime.groupby('PdDistrict').agg({'Category':'count'}).sort_values(by='Category', ascending=False)

,Category
PdDistrict,
southern,157182
mission,119908
northern,105296
bayview,89431
central,85460
tenderloin,81809
ingleside,78845
taraval,65596
park,49313


The Southern PD does not have rest (excessivly dealing with criminals)

In [25]:
crime['Resolution'].unique()

array(['arrest, booked', 'none', 'arrest, cited', 'psychopathic case',
       'juvenile booked', 'unfounded', 'exceptional clearance', 'located',
       'cleared-contact juvenile for more info', 'not prosecuted',
       'juvenile diverted', 'complainant refuses to prosecute',
       'juvenile admonished', 'juvenile cited',
       'district attorney refuses to prosecute',
       'prosecuted by outside agency', 'prosecuted for lesser offense'],
      dtype=object)

#### In each district, how many unresolved cases do we have?

In [28]:
pd_vs_resol = crime.groupby(['PdDistrict','Resolution']).agg({'Category':'count'}).sort_values(by='Category', ascending=False).reset_index()
pd_vs_resol

,PdDistrict,Resolution,Category
0,southern,none,93924
1,northern,none,70251
2,mission,none,63056
3,central,none,60388
4,bayview,none,51785
...,...,...,...
164,northern,prosecuted for lesser offense,6
165,southern,prosecuted for lesser offense,5
166,tenderloin,prosecuted for lesser offense,4
167,central,prosecuted for lesser offense,3


In [29]:
pd_vs_resol[pd_vs_resol['Resolution']=='none']

,PdDistrict,Resolution,Category
0,southern,none,93924
1,northern,none,70251
2,mission,none,63056
3,central,none,60388
4,bayview,none,51785
5,ingleside,none,51047
6,taraval,none,45018
10,richmond,none,32729
11,park,none,30895
12,tenderloin,none,27697


The southern district has the most number of unresolved cases

#### How many crimes resulted in an arrest and citation?

In [31]:
crime.groupby('Resolution').agg({'Category':'count'}).sort_values(by='Category',ascending=False)

,Category
Resolution,
none,526790
"arrest, booked",206403
"arrest, cited",77004
located,17101
psychopathic case,14534
unfounded,9585
juvenile booked,5564
complainant refuses to prosecute,3976
district attorney refuses to prosecute,3934


The individuals involved in 77004 crimes were arrested and issued a citation